The first step to solving the temporal instanton QCQP is to translate the problem by a vector $x:Ax=b$. Of course, there are infinitely many vectors that satisfy this requirement. The question is, is there one vector that simultaneously satisfies $Ax=b$ for every line's QCQP?

Most of the $A$ matrix is fixed from one QCQP to the next. There are only $k$ rows that change, where $k$ is the number of time steps. These are the rows that define auxiliary angle variables in terms of nodal angle variables. Each auxiliary angle variable has the form $a(\theta_i-\theta_k)$, where $a$ is some scaling coefficient. Thus, elements of $b$ corresponding to the $k$ rows in question are all zero. This means we can fix the auxiliary angles to zero and still satisfy $Ax=b$. Because the $k$ auxiliary angle variables may be fixed to zero, we can find a translation point before looping through the lines; there is no need to find a new translation point each time.

I modified my code to pre-compute x_star and pass it to the QCQP solver as an argument. I then commented out the part of the QCQP solver that finds x_star. While doing this, I noticed that the entire problem may be translated ahead of time. I also noticed that the results are not the same. What's going on?

$x^*$ must satisfy two conditions: $Ax^*=b$ and $x^*_3=0$.

$$
\begin{align*}
A_1x_1^* &= b_1 \\
A_2x_2^* &= b_2 \\
A_3x_3^* &= 0
\end{align*}
$$

We can easily set $x_3=0$. This satisfies the third equation, which is the only one that depends on our choice of line. So we ought to be able to solve the remaining two equations before looping through lines.

## Testing

In [1]:
addprocs(2) # vary number of concurrent processes here
@everywhere include("../src/TemporalInstanton.jl")
@everywhere include("../src/mat2tmpinst.jl")
@everywhere using TemporalInstanton

  likely near /home/jkersulis/.julia/v0.4/GraphLayout/src/draw.jl:3
 in depwarn at deprecated.jl:73
 in call at deprecated.jl:50
 in include at ./boot.jl:261
 in include_from_node1 at ./loading.jl:304
 in include at ./boot.jl:261
 in include_from_node1 at ./loading.jl:304
 in require at ./loading.jl:237
 in include at ./boot.jl:261
 in include_from_node1 at ./loading.jl:304
 in include at ./boot.jl:261
 in include_from_node1 at ./loading.jl:304
 in eval at ./sysimg.jl:14
 in anonymous at multi.jl:1350
while loading /home/jkersulis/.julia/v0.4/GraphLayout/src/draw.jl, in expression starting on line 3
  likely near /home/jkersulis/.julia/v0.4/GraphLayout/src/draw.jl:23
  likely near /home/jkersulis/.julia/v0.4/GraphLayout/src/draw.jl:135
  likely near /home/jkersulis/.julia/v0.4/GraphLayout/src/draw.jl:3
 in depwarn at deprecated.jlWARNING: Base.String is deprecated, use AbstractString instead.
  likely near /home/jkersulis/.julia/v0.4/GraphLayout/src/draw.jl:3
73
 in depwarn at deprecat

In [1]:
include("../src/TemporalInstanton.jl")
using TemporalInstanton

In [2]:
# compile everything with this run:
inputData = load_rts96_data(return_as_type=true);
# Thermal model parameters:
inputData.Tamb = 35. # C
inputData.T0 = 60. #46. # initial line steady-state temp

inputData.time_values = 0:30:300 # five minutes in 30-sec steps
inputData.int_length = 300. # seconds = 5 min
Gp,Dp,Rp = inputData.G0,inputData.D0,inputData.R0
inputData.G0 = [0.7*Gp;0.7*Gp;0.7*Gp;0.7*Gp;0.7*Gp;0.7*Gp]
inputData.D0 = [0.9*Dp;0.9*Dp;0.9*Dp;0.9*Dp;0.9*Dp;0.9*Dp]
inputData.R0 = [Rp;1.1*Rp;1.2*Rp;1.3*Rp;1.4*Rp;1.5*Rp]

@time results = solve_temporal_instanton(inputData);

n = length(inputData.k)
nr = length(inputData.Ridx)
T = convert(Int64,length(inputData.G0)/n)
outputData = process_instanton_results(results,n,nr,T,return_as_type=true);

sort(outputData.score)

r=0 check: 	removing 1 lines
ISF pre-check: 	removing lines Int64[]
  

104-element Array{Tuple{Float64,Int64},1}:
 (0.263787830020006,25)  
 (0.3273164183285288,30) 
 (1.2886107366327355,69) 
 (2.2041515936832066,89) 
 (2.322973886040895,118) 
 (2.42316154712363,64)   
 (4.67875546231252,42)   
 (4.865240811106734,102) 
 (4.899167201158152,31)  
 (5.678266048785588,80)  
 (5.938620037616521,1)   
 (6.439172990446961,100) 
 (6.4608887584718095,62) 
 ⋮                       
 (178.0855817862111,99)  
 (303.00328573243416,103)
 (370.61443512027347,66) 
 (370.61443512027347,67) 
 (391.3070873721824,33)  
 (428.94449223314234,40) 
 (923.97998538764,72)    
 (979.1988880469704,117) 
 (1319.8524187115077,79) 
 (1390.0270570261027,110)
 (1967.2698882361206,73) 
 (1967.2698882361206,74) 

9.178186 seconds (9.16 M allocations: 1.218 GB, 1.96% gc time)


In [17]:
size(rand(5,3),2)

3

In [3]:
using JLD
Apart = load("Apart.jld")
Qobj,A,T,b = Apart["Qobj"],Apart["A"],Apart["T"],Apart["b"]

A1,A2,idx1,idx2,idx3 = TemporalInstanton.partition_A(A,Qobj,T)

x_star = load("xstar.jld")["x_star"]

558-element Array{Float64,1}:
   -0.318777 
  -18.1368   
   36.3913   
   79.5597   
   -7.08758  
  -15.057    
    0.54152  
  -93.5651   
   -2.51313  
  226.227    
  383.593    
 -866.299    
   78.9539   
    ⋮        
   -0.14828  
   -0.222527 
    0.0285971
   -0.0473885
    0.0817449
    0.0      
    0.0      
    0.0      
    0.0      
    0.0      
    0.0      
    0.0      

In [7]:
idx2

444-element Array{Int64,1}:
  19
  20
  21
  22
  23
  24
  25
  26
  27
  28
  29
  30
  31
   ⋮
 541
 542
 543
 544
 545
 546
 547
 548
 549
 550
 551
 552

In [16]:
n = size(A,2)
x = zeros(n)

x[idx2] = (b'*A2/(A2'*A2))'[:]
x

558-element Array{Float64,1}:
   0.0      
   0.0      
   0.0      
   0.0      
   0.0      
   0.0      
   0.0      
   0.0      
   0.0      
   0.0      
   0.0      
   0.0      
   0.0      
   ⋮        
  -0.198504 
  -0.253345 
  -0.195784 
  -0.0732502
  -0.208325 
 -19.1781   
   0.0      
   0.0      
   0.0      
   0.0      
   0.0      
   0.0      

In [6]:
(b'*A2/(A2'*A2))'[:]

444-element Array{Float64,1}:
   2.35215e-12
   0.00105911 
  -0.0514843  
   0.0114559  
   0.0186887  
   0.0383686  
   0.0676026  
   0.0674405  
  -0.0252896  
   0.00121349 
  -0.0913014  
  -0.10344    
  -0.134402   
   ⋮          
  -0.141204   
  -0.13954    
  -0.178547   
  -0.189578   
  -0.145823   
  -0.172711   
  -0.198504   
  -0.253345   
  -0.195784   
  -0.0732502  
  -0.208325   
 -19.1781     

In [4]:
[idx1;idx2]

552-element Array{Int64,1}:
   1
   2
   3
   4
   5
   6
   7
   8
   9
  10
  11
  12
  13
   ⋮
 541
 542
 543
 544
 545
 546
 547
 548
 549
 550
 551
 552

In [6]:
[A1 A2]\b

552-element Array{Float64,1}:
   -0.318777  
  -18.1368    
   36.3913    
   79.5597    
   -7.08758   
  -15.057     
    0.54152   
  -93.5651    
   -2.51313   
  226.227     
  383.593     
 -866.299     
   78.9539    
    ⋮         
   -0.088843  
   -0.0671526 
   -0.116673  
   -0.129224  
   -0.00869725
    0.0207089 
   -0.14828   
   -0.222527  
    0.0285971 
   -0.0473885 
    0.0817449 
    0.0       

In [ ]:
using PyPlot
PyPlot.svg(true)

t = load("../data/timing.jld")["timeVecs"]

tTrans    = [ti[1] for ti in t]*1e3
tKern     = [ti[2] for ti in t]*1e3
tEig      = [ti[3] for ti in t]*1e3
tRot      = [ti[4] for ti in t]*1e3
tSec      = [ti[5] for ti in t]*1e3
tMap      = [ti[6] for ti in t]*1e3
misc = outputData.linetimes*1e3 - (tMap+tSec+tRot+tEig+tKern+tTrans)

xvals = collect(1:length(tTrans))

fig = figure(figsize=(11,5))
subplot(1,2,1)
title("RTS-96 QCQP sol. time breakdown")
axis([1;length(xvals);0;50])
xlabel("line index")
ylabel("time (ms)")

bar(xvals, tTrans, color = "r", linewidth=0, width=0.9)
bar(xvals, tKern,  color = "g", bottom = tTrans, linewidth=0, width=0.9)
bar(xvals, tEig,   color = "b", bottom = tKern+tTrans, linewidth=0, width=0.9)
bar(xvals, tRot,   color = "c", bottom = tEig+tKern+tTrans, linewidth=0, width=0.9)
bar(xvals, tSec,   color = "m", bottom = tRot+tEig+tKern+tTrans, linewidth=0, width=0.9)
bar(xvals, tMap,   color = "y", bottom = tSec+tRot+tEig+tKern+tTrans, linewidth=0, width=0.9)
bar(xvals, misc,   color = "k", bottom = tMap+tSec+tRot+tEig+tKern+tTrans, linewidth=0,width=0.9, alpha=0.5)

labels = ["Trans.";
    "Kernel rot.";
    "Eig";
    "Rot.";
    "Secular";
    "Map back";
    "Misc"]

legend(labels,loc=1,ncol=1,fontsize=9,
bbox_to_anchor=(1.32, 1.0))

subplot(1,2,2)
title("Mean over all 104 lines")
sizes = [mean(tTrans);
    mean(tKern);
    mean(tEig);
    mean(tRot);
    mean(tSec);
    mean(tMap);
    mean(misc)]
scale!(sizes,1/sum(sizes))

colors = ["r";
    "g";
    "b";
    "c";
    "m";
    "y";
    "gray"]

pie(sizes, colors=colors, autopct="%1.1f%%", pctdistance=1.2)
# tight_layout()
# savefig("../images/2015-10-13-timing-analysis-check-one.pdf")